In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# --- CONFIGURACIÓN ESTÉTICA PROFESIONAL ---
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 7)
plt.rcParams['font.size'] = 11

# --- 1. PREPARACIÓN DE DATOS (ETL) ---
def obtener_dataset_maestro():
    """
    Carga y fusiona las tablas aplicando correcciones de duplicados
    para habilitar el análisis multidimensional.
    """
    try:
        # Carga
        df_cli = pd.read_csv('clientes.xlsx - clientes.csv.csv')
        df_det = pd.read_csv('detalle_ventas.xlsx - detalle_ventas.csv.csv')
        df_prod = pd.read_csv('productos.xlsx - productos.csv.csv')
        df_ven = pd.read_csv('ventas.xlsx - ventas.csv.csv')
        
        # Conversión de fechas
        df_ven['fecha'] = pd.to_datetime(df_ven['fecha'])
        
        # Fusión Estratégica
        # 1. Detalle + Ventas (Trae fecha, medio_pago, cliente)
        df = pd.merge(df_det, df_ven, on='id_venta', how='inner')
        # 2. + Productos (Trae categoria) - Usamos suffix para evitar colisión de nombres
        df = pd.merge(df, df_prod, on='id_producto', how='left', suffixes=('', '_prod_dup'))
        # 3. + Clientes (Trae ciudad)
        df = pd.merge(df, df_cli, on='id_cliente', how='left', suffixes=('', '_cli_dup'))
        
        return df
    except Exception as e:
        print(f"❌ Error en la preparación de datos: {e}")
        return None
# --- 2. NUEVAS TAREAS DE ANÁLISIS ---

def tarea_analisis_categorias(df):
    """
    Tarea: Visualizar el rendimiento por Categoría de Producto.
    Responde: ¿Qué rubro aporta más facturación a la empresa?
    """
    print("\n📦 Ejecutando Análisis por Categoría...")
    
    # Agrupación
    ventas_cat = df.groupby('categoria')['importe'].sum().sort_values(ascending=False)
    
    # Visualización
    plt.figure(figsize=(10, 6))
    ax = sns.barplot(x=ventas_cat.index, y=ventas_cat.values, palette='viridis', hue=ventas_cat.index, legend=False)
    
    plt.title('Ingresos Totales por Categoría de Producto', fontsize=16, fontweight='bold')
    plt.ylabel('Facturación ($)')
    plt.xlabel('Categoría')
    
    # Añadir etiquetas de datos encima de las barras
    for i, v in enumerate(ventas_cat.values):
        ax.text(i, v + (v*0.01), f"${v:,.0f}", ha='center', fontweight='bold')
        
    plt.show()

